#### Experiment C
##### Overview
- This notebook performs experiment C of the proposed research. The model are trained on different bytes using Chenggang's dataset and testing of the model is performed using Jimmy's dataset.

##### Description of the Dataset
- The dataset has 200k power traces for training from Chenggang's dataset and 20k traces for testing from Jimmy's dataset, 10k power traces with same key and 10k power traces with different key
- The power traces used for testing were collected wihtout performing any optimizations to the dataset

In [1]:
# importing notebook containig utility functions for model training
import sys
sys.path.append('../utilities/')

from modelZoo import *
from modelTrainingUtility import *

In [2]:
# input parameters for training not used for testing required for reading the data from the disk
input_params = {
    "input_path": '../../TripletPower-data/xmega-unmasked/PC1_CB1_TDX1_K1_U_20k/PC1_CB1_TDX1_K1_U_20k_0317.npz', # path to load the dataset
    "output_path": '../models/experiment-q/model/', # path to saved model
    "verbose": True,
    "target_byte": 0, # byte for which ranking is to be performed
    "network_type": "cnn2", # choices={'mlp', 'cnn', 'cnn2', 'wang'}
    "start_idx": 1800,
    "end_idx": 2800,
    "verbose": False,
    "testType": "diffkey", # choices={'samekey', 'diffkey'}
}

if not os.path.isdir(input_params["output_path"]):
    print('creating directory for saving results ...')
    os.makedirs(input_params["output_path"])
else:
    print('directory available for saving results.')

directory available for saving results.


In [3]:
# directory for saving model
model_dir = os.path.join(input_params["output_path"], 'model')

# model_file_name = 'best_model_cnn2_dataset_MPEG_target1_targetbyte_{}.hdf5'.format(input_params["target_byte"])
model_file_name = 'best_model_cnn2_dataset_PC1_CB1_TDX1_K0_U_200k_targetbyte_{}.hdf5'.format(input_params["target_byte"])
model_path = os.path.join(input_params["output_path"], 'model')
mode_weights = os.path.join(model_path, model_file_name)
print('model file: ', mode_weights)

model file:  ../models/experiment-q/model/model/best_model_cnn2_dataset_PC1_CB1_TDX1_K0_U_200k_targetbyte_0.hdf5


In [4]:
# Load profiling and attack data and metadata from the ASCAD database
# val_traces, val_label, val_textin, key
X_attack, Y_attack, plaintext_attack, key = load_test_data_2(input_params)

loading the test data ...
processing data for key byte 0


TypeError: ufunc 'bitwise_xor' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
# Load model
start_idx, end_idx = input_params["start_idx"], input_params["end_idx"]
input_shape = (end_idx-start_idx, 1)
K.clear_session()

model = cnn_best2(input_shape)

print('loading weights...')
model.load_weights(mode_weights)
print('weights loaded successfully!')

In [ ]:
# run the accuracy test
score, accuracy = model.evaluate(X_attack, Y_attack, verbose=input_params["verbose"])
print('test accuracy is: {:f}'.format(accuracy))

In [ ]:
# We test the rank over traces of the Attack dataset, with a step of 10 traces
print('start computing rank value...')
min_trace_idx = 0
max_trace_idx = 10
rank_step = 1
target_byte = input_params["target_byte"]
ranks = full_ranks(model, X_attack, key, plaintext_attack, min_trace_idx, max_trace_idx, target_byte, rank_step)

In [ ]:
# We plot the results
# f_ranks[i] = [t, real_key_rank]
x = [ranks[i][0] for i in range(0, ranks.shape[0])]
y = [ranks[i][1] for i in range(0, ranks.shape[0])]

In [ ]:
testType = input_params["testType"]
fig_save_name =  os.path.join(input_params["output_path"], 'xmega-no-opt' + '_rank_performance_byte_{}_{}.png'.format(target_byte, testType))
print('figure save to file: {}'.format(fig_save_name))
model_file_name = os.path.basename(mode_weights).split('.')[0]

In [ ]:
plot_figure(x, y, model_file_name, 'xmega-no-opt', fig_save_name, testType)